<a href="https://colab.research.google.com/github/JaroslavHolecek/Teaching/blob/master/JupyterNotebook/SQL/VnoreneDotazy_like_in_between_zadani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vnořené dotazy, LIKE, IN, BETWEEN
V tomto notebooku se podíváme na další možnosti jak upřesnit výpis z tabulky.

Aby nám kód fungoval musíme si naistalovat knihovnu mysql.connector

In [ ]:
!pip install mysql.connector

In [3]:
# Vytvoření databáze, se kterou budeme pracovat

import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)

mycursor = mydb.cursor()

try:
    mycursor.execute("""DROP TABLE Automobil;""")
    mydb.commit()
except mysql.connector.Error:
    print("Tato tabulka neexistuje.")

# Vytvoříme tabulku Automobil
mycursor.execute("""CREATE TABLE Automobil(
    id int,
    spz text,
    znacka text,
    pocet_sedadel int,
    max_rychlost int,
    nosnost int,
    nutna_kvalifikace text,
    datum_vyroby date
)""")

mydb.commit()

# Zapíšeme do ni
mycursor.execute("""INSERT INTO Automobil
(id, spz, znacka, pocet_sedadel, max_rychlost, nosnost,  nutna_kvalifikace, datum_vyroby) VALUES
(1, '1A1111', 'Skoda', 4, 150, 3, 'B',  '2000-09-09'),
(2, '2B2222', 'Ford', 2, 220, 2, 'B',  '2020-01-01'),
(3, '3B3333', 'Audi', 4, 220, 2, 'B',  '2018-01-01'),
(4, '4B4444', 'Skoda', 4, 190, 4, 'B',  '2017-05-11');
""")

mydb.commit()

mycursor.close()
mydb.close()

## IN
Používá se spolu s WHERE a umožňuje specifikovat více hodnot zvoleného atributu, podle kterého chceme z tabulky řádky vypsat. Operátor IN je v podstatě zkrácením OR.

Použití IN může vypadat následovně:

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)
mycursor = mydb.cursor()

# Tento příkaz vypíše vše o autech značky Skoda a Audi
mycursor.execute("""SELECT * FROM Automobil WHERE znacka IN ('Skoda', 'Audi')
""")

myresult = mycursor.fetchall()

for id, spz, znacka, pocet_sedadel, max_rychlost, nosnost,  nutna_kvalifikace, datum_vyroby in myresult:
    print(f"""
    Auto má id {id}, spz {spz}, značka {znacka}, má {pocet_sedadel} sedadla, 
    max rychlost {max_rychlost} km/h, nosnost {nosnost} tun,
    je potřeba řidičský průkaz kategorie {nutna_kvalifikace}, bylo vyrobeno {datum_vyroby}
    """)

mycursor.close()
mydb.close()

## LIKE
LIKE se používá k filtrování textu v záznamu. Používá se spolu s WHERE.

Pro filtraci se používají tzv. wildcards. Právě ty jsou zodpovědné za filtraci.
Logiku filtrování řeší SŘDB.

Dělí se na 2 druhy:
* % - přidá 0+ znaků, které se vyplňí samy na základě filtrace
* _ - přidá přesně jeden znak, který se vyplní sám na základě filtrace

Použití LIKE může vypadat následovně:

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)
mycursor = mydb.cursor()

# Přidání záznamu 
mycursor.execute("""INSERT INTO Automobil
(id, spz, znacka, pocet_sedadel, max_rychlost, nosnost,  nutna_kvalifikace, datum_vyroby) VALUES
(5, '5A5555', 'Ferrari', 2, 250, 3, 'B',  '2005-09-09');
""")

mydb.commit()

# Tento příkaz vypíše znacky aut, která začínají na F
mycursor.execute("""SELECT znacka FROM Automobil WHERE znacka LIKE 'F%'
""") 

myresult = mycursor.fetchall()

for znacka in myresult:
    print(f"""
    Auto má značku {znacka}
    """)

print("=======")

# Tento příkaz vypíše znacky aut, která splňují podmínku znacka = F_r_
mycursor.execute("""SELECT znacka FROM Automobil WHERE znacka LIKE 'F_r_'
""") 

myresult = mycursor.fetchall()

for znacka in myresult:
    print(f"""
    Auto má značku {znacka}
    """)

mycursor.close()
mydb.close()

## BETWEEN
Operátor BETWEEN vybírá hodnoty z daného rozsahu. Hodnoty mohou být čísla, text, nebo datum.

Výpis BETWEEN bude obsahovat první i poslední hodnotu z rozsahu.

Použití BETWEEN může vypadat následovně:

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)
mycursor = mydb.cursor()

# Tento příkaz vypíše id a maximální rychlost aut s rychlostí v rozsahu 160 až 230
mycursor.execute("""SELECT id, max_rychlost FROM Automobil 
WHERE max_rychlost BETWEEN 160 AND 230;
""") 

myresult = mycursor.fetchall()

for id, max_rychlost in myresult:
    print(f"""
    Auto má id {id} a maximální rychlost {max_rychlost}
    """)

print("=========")

# Tento příkaz vypíše id a spz aut s spz v rozsahu 2B2222 až 4B4444
mycursor.execute("""SELECT id, spz FROM Automobil 
WHERE spz BETWEEN '2B2222' AND '4B44440';
""") 

myresult = mycursor.fetchall()

for id, spz in myresult:
    print(f"""
    Auto má id {id} a spz {spz}
    """)

mycursor.close()
mydb.close()

## Vnořené dotazy
Vnořený dotaz není nic jiného, než příkaz SELECT vnořený do jiného příkazu SELECT. Vnořené dotazy využijeme tam, kde potřebujeme nejprve zjistit nějakou informaci a v závislosti na ní zjistit pak informace další. Typickým příkladem by mohl být následující dotaz.

Vnořené dotazy vytvoříme tak, že si problém rozdělíme na několik jednoduchých a ty pak poskládáme. 

Pokud tedy budeme chtít vypsat auta, která mají rychlost vyšší, než je průměrná rychlost všech aut. Kód na tento příkaz může vypadat takto:

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)
mycursor = mydb.cursor()

# Nejdříve si daný problém rozdělíme.
# 1. Chceme zjistit průměrnou rychlost
# 2. A vypsat auta s vyšší rychlostí
mycursor.execute("""SELECT * FROM Automobil WHERE max_rychlost > 
(SELECT AVG(max_rychlost) FROM Automobil)
""") # příkazy v závorkách se provedou jako první

myresult = mycursor.fetchall()

for id, spz, znacka, pocet_sedadel, max_rychlost, nosnost,  nutna_kvalifikace, datum_vyroby in myresult:
    print(f"""
    Auto má id {id}, spz {spz}, značka {znacka}, má {pocet_sedadel} sedadla, 
    max rychlost {max_rychlost} km/h, nosnost {nosnost} tun,
    je potřeba řidičský průkaz kategorie {nutna_kvalifikace}, bylo vyrobeno {datum_vyroby}
    """)

print("========")

# Nebo budeme například k dané podmínce přidat, že chceme pouze auta určité značky
# To by vypadalo takto
mycursor.execute("""SELECT * FROM Automobil WHERE max_rychlost > 
(SELECT AVG(max_rychlost) FROM Automobil) AND znacka = 'Audi'
""")

myresult = mycursor.fetchall()

for id, spz, znacka, pocet_sedadel, max_rychlost, nosnost,  nutna_kvalifikace, datum_vyroby in myresult:
    print(f"""
    Auto má id {id}, spz {spz}, značka {znacka}, má {pocet_sedadel} sedadla, 
    max rychlost {max_rychlost} km/h, nosnost {nosnost} tun,
    je potřeba řidičský průkaz kategorie {nutna_kvalifikace}, bylo vyrobeno {datum_vyroby}
    """)

mycursor.close()
mydb.close()

# Cvičení
Bude následovat pár cvičení na procvičení dnešního tématu.

## Cvičení 1:
Z tabulky Automobil z tohoto Notebooku vypište záznamy, které mají hodnotu atributu datum_vyroby = 2018-01-01, nebo 2005-09-09

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)
mycursor = mydb.cursor()

try:
    # TODO: Zde přijde váš kód ->
    pass
    

except mysql.connector.Error as chyba:
    print("Příkaz byl zadán chybně:\n", chyba)

mycursor.close()
mydb.close()

Referenční řešení pro kontrolu:
<pre>
Auto má id 3, spz 3B3333, značka Audi, má 4 sedadla, 
max rychlost 220 km/h, nosnost 2 tun,
je potřeba řidičský průkaz kategorie B, bylo vyrobeno 2018-01-01      

Auto má id 5, spz 5A5555, značka Ferrari, má 2 sedadla, 
max rychlost 250 km/h, nosnost 3 tun,
je potřeba řidičský průkaz kategorie B, bylo vyrobeno 2005-09-09
</pre>

## Cvičení 2:
Z tabulky Automobil vypište záznamy, které mají v atributu spz písmeno B.

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)
mycursor = mydb.cursor()

try:
    # TODO: Zde přijde váš kód ->
    pass

except mysql.connector.Error as chyba:
    print("Příkaz byl zadán chybně:\n", chyba)

mycursor.close()
mydb.close()

Referenční řešení pro kontrolu:
<pre>
Auto má id 2, spz 2B2222, značka Ford, má 2 sedadla, 
max rychlost 220 km/h, nosnost 2 tun,
je potřeba řidičský průkaz kategorie B, bylo vyrobeno 2020-01-01


Auto má id 3, spz 3B3333, značka Audi, má 4 sedadla, 
max rychlost 220 km/h, nosnost 2 tun,
je potřeba řidičský průkaz kategorie B, bylo vyrobeno 2018-01-01


Auto má id 4, spz 4B4444, značka Skoda, má 4 sedadla, 
max rychlost 190 km/h, nosnost 4 tun,
je potřeba řidičský průkaz kategorie B, bylo vyrobeno 2017-05-11
</pre>

## Cvičení 3:
Z tabulky Automobil vypište záznamy, které jsou (abecedně) mezi znacka = 'Audi' a znacka = 'Ford'

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)
mycursor = mydb.cursor()

try:
    # TODO: Zde přijde váš kód ->
    pass

except mysql.connector.Error as chyba:
    print("Příkaz byl zadán chybně:\n", chyba)

mycursor.close()
mydb.close()

Referenční řešení pro kontrolu:
<pre>
Auto má id 2, spz 2B2222, značka Ford, má 2 sedadla, 
max rychlost 220 km/h, nosnost 2 tun,
je potřeba řidičský průkaz kategorie B, bylo vyrobeno 2020-01-01


Auto má id 3, spz 3B3333, značka Audi, má 4 sedadla, 
max rychlost 220 km/h, nosnost 2 tun,
je potřeba řidičský průkaz kategorie B, bylo vyrobeno 2018-01-01


Auto má id 5, spz 5A5555, značka Ferrari, má 2 sedadla, 
max rychlost 250 km/h, nosnost 3 tun,
je potřeba řidičský průkaz kategorie B, bylo vyrobeno 2005-09-09
</pre>

## Cvičení 4:
Z tabulky Automobil vypište záznamy, které mají nosnost menší než je průměrná nosnost, nebo atribut znacka = 'Ferrari'

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)
mycursor = mydb.cursor()

try:
    # TODO: Zde přijde váš kód ->
    pass


except mysql.connector.Error as chyba:
    print("Příkaz byl zadán chybně:\n", chyba)

mycursor.close()
mydb.close()

Referenční řešení pro kontrolu:
<pre>
Auto má id 2, spz 2B2222, značka Ford, má 2 sedadla, 
max rychlost 220 km/h, nosnost 2 tun,
je potřeba řidičský průkaz kategorie B, bylo vyrobeno 2020-01-01


Auto má id 3, spz 3B3333, značka Audi, má 4 sedadla, 
max rychlost 220 km/h, nosnost 2 tun,
je potřeba řidičský průkaz kategorie B, bylo vyrobeno 2018-01-01


Auto má id 5, spz 5A5555, značka Ferrari, má 2 sedadla, 
max rychlost 250 km/h, nosnost 3 tun,
je potřeba řidičský průkaz kategorie B, bylo vyrobeno 2005-09-09
</pre>